In [1]:
#Notebook's sandbox
#Fernando Wario
#May 2017

%matplotlib notebook

import csv
import cv2
import datetime
import glob
import json
import math
import numpy as np
import os
import pandas
import sys
import scipy.cluster
import matplotlib.mlab as mlab
import matplotlib.cm as cm
import matplotlib.image as mpimg
import matplotlib.pyplot as plt

from mpl_toolkits.mplot3d import Axes3D
from joblib import Parallel, delayed
from sklearn import preprocessing
from sklearn.linear_model import TheilSenRegressor
from skimage.transform import radon, rescale
from scipy.stats import multivariate_normal
from scipy import stats

sys.path.append('../functions/')
sys.path.append('../utils/')

from DoG import DoG
from VidPlayer import VidPlayer
from file_manager import File_manager
from wdd_decoder_functions import filter_with_kernel, calc_accumulator, calc_angles, read_DD, def_orientation, subst_angles, read_DDvector

#Radian
radi = 180/np.pi

In [9]:
#GROUND TRUTH DATA
#Path to the Ground Truth data
pathGTData = '../Data/GTData'
folders = os.listdir(pathGTData)

#Ground Truth data is loaded
GTArray = {}

for i in folders:
    #Path to next GT file
    inputFile = os.path.join(pathGTData,i)
    print('working on ' + inputFile)
    GTFile = open(inputFile, 'rt', encoding='utf-8-sig')
    for row in csv.reader(GTFile, delimiter = ','):
        key = row[0]
        #If it is the first time this WR is registered
        if (key not in GTArray):
            GTArray[key] = [float(row[-1])]
        #If not, it is add
        else:
            GTArray[key].append(float(row[-1]))

#DECODER DATA
#Path to the Decoder data
inputFile = '../Data/GTRuns_decoder.csv'

#Decoder data is loaded
DecArray = {}
with open(inputFile, 'rt', encoding='utf-8-sig') as DecFile:
    reader = csv.reader(DecFile, delimiter = ',')
    next(reader)

    for row in reader:    
        key = row[0]    
        DecArray[key] = float(row[2])*radi
    print('done with DecArray!')

#DOT DETECTOR DATA
#Path to the Dot Detector data
path = '../Data/GTRuns/'

#Dot Detector data is loaded
DDArray = {}
DDArray = read_DD(path)
print('done with DDArray!')

#ADJUSTED DECODER DATA WITH NEW def_orientation function
AdjArray = {}
AdjArray = def_orientation(DecArray, DDArray)
print('done with AdjArray!')

#Average and error data is computed
#Average Ground Truth
AvgGTArray = {}
#Error Decoder
ErrDecArray = {}
#Error Adjusted Decoder
ErrAdjArray = {}
for v in GTArray:
    #Average Ground Truth is computed using circular statistics with appropriate range
    AvgGTArray[v] = stats.circmean(GTArray[v], high=180, low=-180)
    #Error Decoder is computed using the function subst_angles()
    ErrDecArray[v] = subst_angles(AvgGTArray[v],DecArray[v])
    #Error Adjusted Decoder is computed using the function subst_angles()
    ErrAdjArray[v] = subst_angles(AvgGTArray[v],AdjArray[v])
print('done with AvGTArray, ErrDecArray and ErrAdjArray')

working on ../Data/GTData\20170504_1318.csv
working on ../Data/GTData\20170522_1219.csv
working on ../Data/GTData\20170522_1418.csv
working on ../Data/GTData\20170522_1731.csv
working on ../Data/GTData\20170529_1449.csv
working on ../Data/GTData\20170529_1546.csv
working on ../Data/GTData\20170613_1108.csv
working on ../Data/GTData\20170619_1229.csv
done with DecArray!
done with DDArray!
done with AdjArray!
done with AvGTArray, ErrDecArray and ErrAdjArray


In [10]:
#ErrDecArray distribution
lim = 150

#For ErrDecArray
#An array containing all keys to WRuns with errors over the limit is generated
ErrDecKey = [key for key, value in ErrDecArray.items() if np.max(np.abs(value)) > lim]
ErrDecKey.sort()
#and the errors under that limit
list_values = [value for key, value in ErrDecArray.items() if np.max(np.abs(value)) <= lim]

list_values = np.array(list_values)

print(len(list_values))

mean=np.mean(list_values)
sigma=np.std(list_values)

print('For ErrDecArray')
print('mean = ' + str(mean))
print('std = ' + str(sigma))

181
For ErrDecArray
mean = -2.92038091535
std = 7.36883242338


In [11]:
#ErrAdjArray distribution
lim = 150

#For ErrAdjArray3
#An array containing all keys to WRuns with errors over the limit is generated
ErrAdjKey = [key for key, value in ErrAdjArray.items() if np.max(np.abs(value)) > lim]
ErrAdjKey.sort()
#and the errors under that limit
list_values = [value for key, value in ErrAdjArray.items() if np.max(np.abs(value)) <= lim]

list_values = np.array(list_values)

print(len(list_values))

mean=np.mean(list_values)
sigma=np.std(list_values)

print('For ErrAdjArray')
print('mean = ' + str(mean))
print('std = ' + str(sigma))

181
For ErrAdjArray
mean = -2.92038091535
std = 7.36883242338


In [12]:
################# Displays WR's as videos ##############
#To skip to the next dance press "Y"
#To close the window press "Esc"
#Path to the Waggle runs
path = '../Data/GTRuns/'
length = len(ErrAdjKey)

#control variables initialized
counter = 0
next = 0
first_frame = True

while (counter < length and next != -1):
    key = ErrAdjKey[counter]
    pathWR = (path + key + '/')
    #vp object is initialized
    vp = VidPlayer()
    #waggle run images
    vp.set_vid_path(pathWR + 'image_%03d.png')
    capture = vp.get_vid_capture()
    #The video is played in a loop
    while(vp.get_vid_running()):
        ret, image = capture.read()
        if (ret == True):
            #image is resized
            image = cv2.resize(image, None, fx = 8, fy = 8, interpolation = cv2.INTER_CUBIC)
            #Info is displayed in the video window
            #Path is displayed
            cv2.putText(image,pathWR,(10,20), cv2.FONT_HERSHEY_SIMPLEX, .5,(255,255,255),1,cv2.LINE_AA)
            #Counter is displayed
            cv2.putText(image,'WR ' + str(counter + 1),(366,16), cv2.FONT_HERSHEY_SIMPLEX, .25,(255,255,255),1,cv2.LINE_AA)
            #Angles are displayed
            cv2.putText(image,'GT_angle  = %5.4f' %float(AvgGTArray[key]),(10,336), cv2.FONT_HERSHEY_SIMPLEX, .5,(255,255,255),1,cv2.LINE_AA)
            cv2.putText(image,'Dec_angle = %5.4f' %float(DecArray[key]),(10,354), cv2.FONT_HERSHEY_SIMPLEX, .5,(255,255,255),1,cv2.LINE_AA)
            cv2.putText(image,'DD_angle  = %5.4f' %float(DDArray[key]),(10,372), cv2.FONT_HERSHEY_SIMPLEX, .5,(255,255,255),1,cv2.LINE_AA)
            cv2.putText(image,'Adj_angle  = %5.4f' %float(AdjArray[key]),(10,390), cv2.FONT_HERSHEY_SIMPLEX, .5,(255,255,255),1,cv2.LINE_AA)
            cv2.imshow("Waggle Run", image)
            #Call to video player controls
            vp.vid_player_controlls()
        else:
            capture = vp.vid_restart()            
    first_frame = True
    next = vp.get_vid_dance()
    counter += 1

    
    #Window is detroyed
capture.release()
cv2.destroyAllWindows()

In [ ]:
################# Sandbox's sandbox #######################################

###########################################################################

#Save data in csv and json files
#datetime is used as reference to name the file
dt = datetime.datetime.now()

#All GTRuns with GT_angle, Dec_angle, DDvector
if (os.path.exists('../Data/' + str(dt.strftime('%Y%m%d_%H%M')) + '_GTRuns.csv')):
    os.remove('../Data/' + str(dt.strftime('%Y%m%d_%H%M')) + '_GTRuns.csv')

with open('../Data/' + str(dt.strftime('%Y%m%d_%H%M')) + '_GTRuns.csv', 'at', newline='') as out_file:
    writer = csv.writer(out_file, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for key in sorted(AvgGTArray.keys()):
        # key, GT_angle, Dec_angle, DDvector
        writer.writerow([key, AvgGTArray[key], DecArray[key], read_DDvector(path + key)])

with open('../Data/' + str(dt.strftime('%Y%m%d_%H%M')) + '_GTRuns.json', 'w') as out_file:
    for key in sorted(AvgGTArray.keys()):
        # key, GT_angle, Dec_angle, DDvector
        json.dump([key, AvgGTArray[key], DecArray[key], read_DDvector(path + key)], out_file)
        

#All GTRuns with GT_angle, Dec_angle, DDAngle, AdjAngle
if (os.path.exists('../Data/' + str(dt.strftime('%Y%m%d_%H%M')) + '_OutlayersGTRuns.csv')):
    os.remove('../Data/' + str(dt.strftime('%Y%m%d_%H%M')) + '_OutlayersGTRuns.csv')

with open('../Data/' + str(dt.strftime('%Y%m%d_%H%M')) + '_OutlayersGTRuns.csv', 'at', newline='') as out_file:
    writer = csv.writer(out_file, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for key in ErrDecKey:
        # key, GT_angle, Dec_angle, DD_angle, Adj_angle
        writer.writerow([key, AvgGTArray[key], DecArray[key], DDArray[key], AdjArray[key]])

###########################################################################        
        
#Prints the angles info to visualize errors in the DecArray
for i in ErrDecKey:
    print(i)
    print ('GTArray:       ' + str(GTArray[i]))
    print ('DecArray:      ' + str(DecArray[i]))
    print ('AvgGTArray:    ' + str(AvgGTArray[i]))
    print ('ErrDecArray:   ' + str(ErrDecArray[i]))
    print ('DDArray:       ' + str(DDArray[i]))

###########################################################################

#Angles are displayed
    cv2.putText(image,'GT_angle  = %5.4f' %float(csvFileArray[counter][1]),(10,354), cv2.FONT_HERSHEY_SIMPLEX, .5,(255,255,255),1,cv2.LINE_AA)
    cv2.putText(image,'Dec_angle = %5.4f' %float(csvFileArray[counter][2]),(10,372), cv2.FONT_HERSHEY_SIMPLEX, .5,(255,255,255),1,cv2.LINE_AA)
    cv2.putText(image,'DD_angle  = %5.4f' %float(csvFileArray[counter][3]),(10,390), cv2.FONT_HERSHEY_SIMPLEX, .5,(255,255,255),1,cv2.LINE_AA)

###########################################################################
            
#Path to the Ground Truth data
path = '../Data/GTData'
folders = os.listdir(path)

#Ground Truth data is loaded
GTArray = {}

for i in folders:    
    #Path to next GT file
    inputFile = os.path.join(path,i)
    print('working on ' + inputFile)
    GTFile = open(inputFile, 'rt', encoding='utf-8-sig')
    for row in csv.reader(GTFile, delimiter = ','):
        key = row[0]
        #If it is the first time this WR is registered
        if (key not in GTArray):
            GTArray[key] = [float(row[-1])]
        #If not, it is add
        else:
            GTArray[key].append(float(row[-1]))

###########################################################################            
            
#IF IT HAPPENS THAT YOU WANT TO DISPLAY JUST ONE VIDEO AT A TIME
path = '../Data/GTRuns/'
key = '20160814_1003_1/14'
#control variables initialized
counter = 0
next = 0
first_frame = True

while (next != -1):
    pathWR = (path + key + '/')
    #vp object is initialized
    vp = VidPlayer()
    #waggle run images
    vp.set_vid_path(pathWR + 'image_%03d.png')
    capture = vp.get_vid_capture()
    #The video is played in a loop
    while(vp.get_vid_running()):
        ret, image = capture.read()
        if (ret == True):
            #image is resized
            image = cv2.resize(image, None, fx = 8, fy = 8, interpolation = cv2.INTER_CUBIC)
            #Info is displayed in the video window
            #Path is displayed
            cv2.putText(image,pathWR,(10,20), cv2.FONT_HERSHEY_SIMPLEX, .5,(255,255,255),1,cv2.LINE_AA)            
            #Angles are displayed
            cv2.putText(image,'GT_angle  = %5.4f' %float(AvgGTArray[key]),(10,336), cv2.FONT_HERSHEY_SIMPLEX, .5,(255,255,255),1,cv2.LINE_AA)
            cv2.putText(image,'Dec_angle = %5.4f' %float(DecArray[key]),(10,354), cv2.FONT_HERSHEY_SIMPLEX, .5,(255,255,255),1,cv2.LINE_AA)
            cv2.putText(image,'DD_angle  = %5.4f' %float(DDArray3[key]),(10,372), cv2.FONT_HERSHEY_SIMPLEX, .5,(255,255,255),1,cv2.LINE_AA)
            cv2.putText(image,'Adj_angle  = %5.4f' %float(AdjArray3[key]),(10,390), cv2.FONT_HERSHEY_SIMPLEX, .5,(255,255,255),1,cv2.LINE_AA)
            cv2.imshow("Waggle Run", image)
            #Call to video player controls
            vp.vid_player_controlls()
        else:
            capture = vp.vid_restart()            
    first_frame = True
    next = vp.get_vid_dance()
    counter += 1
#Window is detroyed
capture.release()
cv2.destroyAllWindows()

###########################################################################            
#To generate data for homography test from Tim
def read_homography(WRPath):    
    WRPath = str(WRPath + '/')
    #path to the waggle run
    split_WR = WRPath.split("/")    
    #reads the csv file containing coordinates of dancing bee
    with open(WRPath + split_WR[-3] + "_" + split_WR[-2] + '.csv', 'rt', encoding='utf-8-sig') as file:    
        #Reading third row
        reader = csv.reader(file)
        dd_float = [row for idx, row in enumerate(reader) if idx == 2]        
    return dd_float


#DOT DETECTOR DATA
#Path to the Dot Detector data
path = '../Data/GTRuns/'

#It writes in destination file only real positive dances
origin = '../Data/GTRuns_decoder.csv'
destination = '../Data/GTRuns_homography.csv'

#Destination file is removed if it already exists
if (os.path.exists(destination)):
    os.remove(destination)

#Opens index file with the filter information
with open(origin, 'rt', encoding='utf-8-sig') as file1:
    #reader1 = csv.reader(file1, delimiter=',', quotechar='|')
    reader1 = csv.reader(file1, delimiter=',')
    for line in reader1:
        with open(destination, 'at', newline='') as out_file:
            writer = csv.writer(out_file, delimiter=',', quoting=csv.QUOTE_MINIMAL)
            #key and angle are written in destination file
            row = line[:-1]
            row.extend(read_homography(path+line[0]))
            writer.writerow(row)
